In [1]:
from legion.toolchain.metrics import Metric
from legion.toolchain import model


In [2]:
model.init('recognize digits', '1.0')

In [3]:
# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics

# The digits dataset
digits = datasets.load_digits()

# The data that we are interested in is made of 8x8 images of digits, let's
# have a look at the first 4 images, stored in the `images` attribute of the
# dataset.  If we were working from image files, we could load them using
# matplotlib.pyplot.imread.  Note that each image must have the same size. For these
# images, we know which digit they represent: it is given in the 'target' of
# the dataset.
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples // 2], digits.target[:n_samples // 2])

# Now predict the value of the digit on the second half:
expected = digits.target[n_samples // 2:]
predicted = classifier.predict(data[n_samples // 2:])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

images_and_predictions = list(zip(digits.images[n_samples // 2:], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

plt.show()

Classification report for classifier SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False):
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        88
           1       0.99      0.97      0.98        91
           2       0.99      0.99      0.99        86
           3       0.98      0.87      0.92        91
           4       0.99      0.96      0.97        92
           5       0.95      0.97      0.96        91
           6       0.99      0.99      0.99        91
           7       0.96      0.99      0.97        89
           8       0.94      1.00      0.97        88
           9       0.93      0.98      0.95        92

   micro avg       0.97      0.97      0.97       899
   macro avg       0.97      0.97      0.97       899
weighted avg       0.97      0.97 

<Figure size 640x480 with 8 Axes>

In [4]:
expected = digits.target[n_samples // 2:]
predicted = classifier.predict(data[n_samples // 2:])
accuracy = metrics.accuracy_score(expected, predicted)

model.send_metric(Metric.TEST_ACCURACY, accuracy * 100)

In [5]:
from PIL import Image 
import numpy as np

In [6]:
def predict(v):
    image_file = v['image'].convert('L')
    image_file = image_file.resize((8, 8), Image.BOX)
    array = np.asarray(image_file, dtype=np.uint8)
    array = list(array.reshape(64))
    array = np.array([(255-x) * 1.0 / 10 for x in array], dtype=np.float)
    return {'digit': int(classifier.predict([array])[0])}


In [7]:
image_file = Image.open('nine.png')
payload = {'image': image_file}

digit = predict(payload)
digit

{'digit': 9}

In [8]:
model.export(lambda x: predict(x), {'image': model.image}).save()

In [9]:
model.show_local_metrics()

,Model ID,Model Version,Last Update,test-accuracy
0,recognize-digits,1.0,05/18/19 21:45:02,96.885428
